# PyCaret Time‑Series Forecasting por Ticker 🕒📈

Este cuaderno crea **un modelo de serie temporal por cada acción** registrada en tu base de datos, prediciendo los próximos **7 días** de:

* `precio_cierre`
* `precio_max`
* `precio_min`
* `volumen_operado`

> ⚠️ **Requisitos**  
> * Base de datos PostgreSQL accesible (usa las variables de entorno `DB_USER`, `DB_PASSWORD`, `DB_HOST`, `DB_NAME`).  
> * Paquetes instalados: `pycaret[time_series]`, `sqlalchemy`, `pandas`, `matplotlib`.  

Ejecútalo cell‑a‑cell para revisar resultados y ajustar hiperparámetros si lo deseas.


In [ ]:
import os
import pandas as pd
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from pycaret.time_series import *
import warnings, itertools, datetime as dt

warnings.filterwarnings('ignore')

# Conexión a la base de datos -----------------------------------------------------------------
USER = os.getenv("DB_USER", "usuario")
PASSWORD = os.getenv("DB_PASSWORD", "contraseña")
HOST = os.getenv("DB_HOST", "db")
DB = os.getenv("DB_NAME", "patagonia_db")

DATABASE_URL = f"postgresql://{USER}:{PASSWORD}@{HOST}/{DB}"
engine = create_engine(DATABASE_URL)

print(f'Conectado a: {DATABASE_URL}')

In [ ]:

# ================================================================
# PARAMETRIZACIÓN DE VARIABLES A PRONOSTICAR
# Cambia True/False según lo que quieras incluir en cada corrida
# ================================================================
PARAMS = {
    'predict_precio_cierre': True,
    'predict_precio_max':    False,
    'predict_precio_min':    False,
    'predict_volumen_operado': False,
}

TARGET_VARS = [var for var, flag in {
    'precio_cierre':        PARAMS['predict_precio_cierre'],
    'precio_max':           PARAMS['predict_precio_max'],
    'precio_min':           PARAMS['predict_precio_min'],
    'volumen_operado':      PARAMS['predict_volumen_operado'],
}.items() if flag]

print('Variables a modelar:', TARGET_VARS)


In [ ]:

import logging, datetime
from pathlib import Path
RUN_TS = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
OUT_DIR = Path(f'outputs_{RUN_TS}')
OUT_DIR.mkdir(parents=True, exist_ok=True)

logging.basicConfig(
    filename=OUT_DIR/'run.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logging.info('Inicio de corrida. Variables: %s', TARGET_VARS)
print('Resultados se guardarán en:', OUT_DIR)


In [ ]:
# Consulta SQL (ajusta si tu esquema cambia) --------------------------------------------------
query = '''
SELECT c.id_empresa,
       e.ticker,
       c.fecha,
       c.precio_apertura,
       c.precio_cierre,
       c.precio_max,
       c.precio_min,
       c.volumen_operado,
       c.variacion_porcentaje
FROM cotizacion_x_empresa c
JOIN empresas e ON c.id_empresa = e.id_empresa
ORDER BY e.ticker, c.fecha;
'''
df = pd.read_sql(query, engine)

# Conversión de tipos -------------------------------------------------------------------------
df['fecha'] = pd.to_datetime(df['fecha'])
for col in ['precio_apertura','precio_cierre','precio_max','precio_min','volumen_operado','variacion_porcentaje']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

print('Dataset shape:', df.shape)
df.head()

In [ ]:
# Parámetros globales -------------------------------------------------------------------------
FH = 22  # 7 días de test + 15 días extra  # horizonte de predicción

### Nota sobre imputación
Se fuerza la frecuencia a `'B'` (días hábiles) cuando no puede inferirse.
Los huecos de fines de semana y feriados se **imputan con forward‑fill**.
Si el primer valor queda vacío se aplica back‑fill. Así evitamos NaNs antes de entrenar.

In [ ]:

def plot_forecast(original: pd.Series, forecast_df: pd.DataFrame, title:str, zoom_window:int=30):
    """Dibuja histórico + forecast con IC 95 % (si existe) y zoom final."""
    import matplotlib.pyplot as plt
    
    preds = forecast_df['y_pred']
    lower = forecast_df.get('y_pred_lower')
    upper = forecast_df.get('y_pred_upper')
    
    # función interna para trazar
    def _draw(ax, hist_idx, subtitle):
        ax.plot(hist_idx, original.loc[hist_idx].values, label='Histórico')
        ax.plot(preds.index, preds.values, linestyle='--', marker='o', label='Pronosticado')
        if lower is not None and upper is not None and lower.notna().all():
            ax.fill_between(preds.index, lower.values, upper.values, alpha=0.25, label='IC 95 %')
        ax.set_title(subtitle)
        ax.grid(True)
        ax.legend()
    
    # gráfico completo
    fig, ax = plt.subplots(figsize=(11,4))
    _draw(ax, original.index, f'{title} – completo')
    fig.savefig(OUT_DIR/f'{title.replace(" / ", "_")}_full.png'); fig.savefig(OUT_DIR/f'{title.replace(" / ", "_")}_zoom.png'); plt.show()
    
    # zoom
    zoom_start = original.index[-zoom_window] if len(original) >= zoom_window else original.index[0]
    fig, ax = plt.subplots(figsize=(11,4))
    _draw(ax, original.loc[zoom_start:].index, f'{title} – últimos {zoom_window} d + forecast')
    plt.show()

In [ ]:

def train_forecast(series: pd.Series, fh:int = 22, sort_metric:str = 'MASE'):
    """Entrena modelo TS, devuelve DataFrame pronóstico + intervalos 95 %."""
    # --- preparar serie ---
    series = series.sort_index()
    freq = pd.infer_freq(series.index) or 'B'
    series = series.asfreq(freq).ffill().bfill()
    
    # --- setup / entrenamiento ---
    exp = setup(
        data=series,
        fh=fh,
        fold=3,
        session_id=123,
        verbose=False
    )
    best = compare_models(sort=sort_metric, verbose=False)
    
    # --- predicción con intervalos 95 % ---
    # cobertura central 95 % => quantiles [0.025, 0.975]
    preds_df = predict_model(best, fh=fh, coverage=[0.025, 0.975])
    
    # Renombrar columnas lower/upper genéricas a estándar --------------------
    lower_cols = [c for c in preds_df.columns if 'lower' in c.lower()]
    upper_cols = [c for c in preds_df.columns if 'upper' in c.lower()]
    
    if lower_cols and upper_cols:
        preds_df = preds_df.rename(columns={lower_cols[0]:'y_pred_lower', upper_cols[0]:'y_pred_upper'})
    else:
        preds_df['y_pred_lower'] = pd.NA
        preds_df['y_pred_upper'] = pd.NA
    
    return best, preds_df


In [ ]:

results = []
tickers = df['ticker'].unique()
for ticker in tickers:
    df_t = df[df['ticker'] == ticker].copy().sort_values('fecha')
    df_t.set_index('fecha', inplace=True)

    print(f'\n================  {ticker}  ================')
    for target in TARGET_VARS:
        series = df_t[target].dropna()
        if len(series) < 40:
            print(f'⚠️  {ticker} - {target}: muy pocos datos, omitiendo.')
            continue

        print(f'→ Entrenando modelo para {ticker} - {target} (n={len(series)})')
        best, forecast_df = train_forecast(series, fh=FH)
        
        for f_date, row in forecast_df.iterrows():
            results.append({
                'ticker': ticker,
                'variable': target,
                'date': f_date,
                'forecast': row['y_pred'],
                'lower': row.get('y_pred_lower', pd.NA),
                'upper': row.get('y_pred_upper', pd.NA)
            })

        plot_forecast(series, forecast_df, f'{ticker} - {target}', zoom_window=30)


In [ ]:
# Resultados finales --------------------------------------------------------------------------
pred_df = pd.DataFrame(results)
pred_df.head(20)

In [ ]:

# Guardar DataFrame y reporte DOCX
pred_df.to_csv(OUT_DIR/'forecast_full.csv', index=False)
logging.info('Se guardó forecast_full.csv con %d filas', len(pred_df))

try:
    from docx import Document
    doc = Document()
    doc.add_heading('Pronóstico de Acciones', level=1)
    doc.add_paragraph(f'Ejecución: {RUN_TS}')

    for ticker in pred_df['ticker'].unique():
        doc.add_heading(ticker, level=2)
        sub = pred_df[pred_df['ticker']==ticker].copy()
        for var in sub['variable'].unique():
            sub2 = sub[sub['variable']==var]
            doc.add_heading(var, level=3)
            # Añadir tabla
            table = doc.add_table(rows=1, cols=4)
            hdr_cells = table.rows[0].cells
            hdr_cells[0].text = 'Fecha'
            hdr_cells[1].text = 'Predicción'
            hdr_cells[2].text = 'Lower'
            hdr_cells[3].text = 'Upper'
            for _,r in sub2.iterrows():
                row_cells = table.add_row().cells
                row_cells[0].text = str(r['date'].date())
                row_cells[1].text = f"{r['forecast']:.2f}"
                row_cells[2].text = '-' if pd.isna(r['lower']) else f"{r['lower']:.2f}"
                row_cells[3].text = '-' if pd.isna(r['upper']) else f"{r['upper']:.2f}"
    doc.save(OUT_DIR/'reporte_proyecciones.docx')
    logging.info('Se generó reporte_proyecciones.docx')
except ImportError:
    logging.warning('python-docx no instalado; omitiendo reporte DOCX')
